In [7]:
import mbuild as mb
import foyer
import hoomd
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield

esp_ff = foyer.Forcefield(forcefield_files="PCPDTFBT_C1_BO.xml")

# We have to add the underscore to the names manually if we are using foyer XML files without SMARTS definitions
mono = mb.load("PCPDTFBT_C1_BO_typed.mol2")
for p in mono.particles():
    p.name = f"_{p.name}"
    
box = mb.fill_box(compound=mono, n_compounds=5, box=[30,30,30])
box_pmd = esp_ff.apply(box)

snapshot, forcefield, refs = create_hoomd_forcefield(box_pmd, auto_scale=True, r_cut=2.5)

/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _H1
  warnings.warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _F1
  warnings.warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _S1
  warnings.warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _C1
  warnings.warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _C2
  warnings.warn(
/Users/madilyn/miniconda3/envs/espaloma/

/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <_C1 pos=([18.5867 14.367  16.8022]), 4 bonds, id: 6218483696> does not have a charge.They will not be accounted for during this calculation.
  warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <_C2 pos=([18.5514 14.4491 16.6749]), 3 bonds, id: 6218482496> does not have a charge.They will not be accounted for during this calculation.
  warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <_C2 pos=([18.5667 14.5999 16.7119]), 3 bonds, id: 6218482928> does not have a charge.They will not be accounted for during this calculation.
  warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <_S1 pos=([18.4609 14.6951 16.5979]), 2 bonds, id: 6218482784>

/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:455: UserWarning: Parameters have been assigned to all proper dihedrals.  However, there are more parameterized dihedrals (18000) than total system dihedrals (3000).  This may be due to having multiple periodic dihedrals for a single system dihedral.
  warnings.warn(msg)
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:349: UserWarning: Parameters have not been assigned to all impropers. Total system impropers: 1140, Parameterized impropers: 0. Note that if your system contains torsions of Ryckaert-Bellemans functional form, all of these torsions are processed as propers
  warnings.warn(msg)
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:926: UserWarning: Parametrized structure has non-zero charge.Structure's total charge: 20.660990000000247
  warnings.warn(


Processing LJ and QQ
Processing 1-4 interactions, adjusting neighborlist exclusions
Processing harmonic bonds
Processing harmonic angles
Processing periodic torsions


In [8]:
#setting cpu and simulation 
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu,seed=0)
sim.create_state_from_snapshot(snapshot)

#setting the integrator
kt = 1.2
free_particle = hoomd.filter.Tags(tags=[0,1])  #letting hoomd know which particles to update the positions of
integrator = hoomd.md.Integrator(dt = 0.01)
nvt = hoomd.md.methods.NVT(filter=hoomd.filter.All(), kT=1.2, tau=0.5) #what is tau?
integrator.forces = forcefield
integrator.methods.append(nvt)
sim.operations.integrator = integrator
sim.state.thermalize_particle_momenta(filter=free_particle, kT=kt)
# Set up GSD writer
gsd_writer = hoomd.write.GSD(
    trigger=hoomd.trigger.Periodic(int(2e2)),
    filename="traj_PCPDTFBT_C1_BO.gsd",  #name the output file
    mode="wb"
)
sim.operations.writers.append(gsd_writer)

In [10]:
sim.run(1e5)

**ERROR**: Particle with unique tag 788 is no longer in the simulation box.

Cartesian coordinates: 
x: 2132.74 y: 2133.2 z: 5078.24
Fractional coordinates: 
f.x: 25.8341 f.y: 25.8396 f.z: 60.8226
Local box lo: (-42.0923, -42.0923, -42.0923)
          hi: (42.0923, 42.0923, 42.0923)


RuntimeError: Error computing cell list